In [109]:
import random
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
true_class, false_class = 'T','F'
# random.seed(1245)

#  Reading the data and converting into DataFrame

In [3]:
read_file = lambda x : [tuple(line.strip().split("\t")) for line in open(x).readlines() if len(line.strip()) > 0]
assign_class = lambda x,y : [l + (y, ) for l in x]

true_lines = assign_class(read_file("./hotelT-train.txt"), true_class)
false_lines = assign_class(read_file("./hotelF-train.txt"), false_class)

all_lines = true_lines + false_lines

random.shuffle(all_lines)

raw_data_pd = pd.DataFrame(all_lines, columns = ["id", "sentence", "class"])

# Tokenizing the sentences, removing stop words

In [4]:
stop_words = set(stopwords.words('english'))
raw_data_pd["sentence_tokens"] = raw_data_pd["sentence"].apply(lambda x : [token.lower() for token in word_tokenize(x) if token.isalpha() and token.lower() not in stop_words])

# K-Fold cross validation with all words as features

In [132]:
raw_data_pd = raw_data_pd.sample(frac=1).reset_index(drop=True)
tokens = raw_data_pd["sentence_tokens"]
labels = raw_data_pd["class"]
kf = StratifiedKFold(n_splits=10)
accuracy = []
for train_idx, test_idx in kf.split(tokens, labels):
    X_train, X_test = [tokens[i] for i in train_idx], [tokens[i] for i in test_idx]
    y_train, y_test = [labels[i] for i in train_idx], [labels[i] for i in test_idx]
    vectorizer = CountVectorizer()
    train_features = vectorizer.fit_transform(map(lambda x : " ".join(x), X_train))
    test_features = vectorizer.transform(map(lambda x : " ".join(x), X_test))
    classifier = MultinomialNB()
    classifier.fit(train_features, y_train)
    predictions = classifier.predict(test_features)
    accuracy.append(np.sum(y_test == predictions)*1.0/ len(y_test))
print np.average(accuracy)

0.478463203463
